In [4]:
import pandas as pd
# On enregistre le path de l'objet (dans un bucket S3)
bucket_path = 'gs://mspr-data-ia-raw/resultats-par-niveau-subcom-t1-france-entiere.xlsx'

# On initie un dataframe avec Pandas
df = pd.read_excel(bucket_path)

# Liste des codes départements de l’Île-de-France (SAUF PARIS)
codes_idf = ['77', '78', '91', '92', '93', '94', '95']

# Filtre sur la colonne "Code du département"
df = df[df["Code du département"].astype(str).isin(codes_idf)]

df.head()

,Code du département,Libellé du département,Code de la commune,Libellé de la commune,Etat saisie,Inscrits,Abstentions,% Abs/Ins,Votants,% Vot/Ins,...,Unnamed: 93,Unnamed: 94,Unnamed: 95,Unnamed: 96,Unnamed: 97,Unnamed: 98,Unnamed: 99,Unnamed: 100,Unnamed: 101,Unnamed: 102
29963,77,Seine-et-Marne,1,Achères-la-Forêt,Complet,920,158,17.17,762,82.83,...,6,0.65,0.81,12,M,DUPONT-AIGNAN,Nicolas,17,1.85,2.28
29964,77,Seine-et-Marne,2,Amillis,Complet,560,121,21.61,439,78.39,...,3,0.54,0.69,12,M,DUPONT-AIGNAN,Nicolas,11,1.96,2.55
29965,77,Seine-et-Marne,3,Amponville,Complet,295,59,20.00,236,80.00,...,2,0.68,0.87,12,M,DUPONT-AIGNAN,Nicolas,2,0.68,0.87
29966,77,Seine-et-Marne,4,Andrezel,Complet,209,25,11.96,184,88.04,...,1,0.48,0.55,12,M,DUPONT-AIGNAN,Nicolas,6,2.87,3.30
29967,77,Seine-et-Marne,5,Annet-sur-Marne,Complet,2188,406,18.56,1782,81.44,...,7,0.32,0.40,12,M,DUPONT-AIGNAN,Nicolas,44,2.01,2.50


In [5]:
from google.cloud import storage

client = storage.Client()
bucket = client.get_bucket('mspr-data-ia-raw')  # Nom du bucket
blob = bucket.blob('IDF_T1_res_presidentielles.csv')   # Nom du nouveau csv

# Sauvegarde locale temporaire
df.to_csv('temp_export.csv', index=False)

# Upload
blob.upload_from_filename('temp_export.csv')

In [6]:
import pandas as pd
# On enregistre le path de l'objet (dans un bucket S3)
bucket_path = 'gs://mspr-data-ia-raw/IDF_T1_res_presidentielles.csv'

# On initie un dataframe avec Pandas
df = pd.read_csv(bucket_path, sep=',')

df.head(5)

print(df.columns.tolist())

['Code du département', 'Libellé du département', 'Code de la commune', 'Libellé de la commune', 'Etat saisie', 'Inscrits', 'Abstentions', '% Abs/Ins', 'Votants', '% Vot/Ins', 'Blancs', '% Blancs/Ins', '% Blancs/Vot', 'Nuls', '% Nuls/Ins', '% Nuls/Vot', 'Exprimés', '% Exp/Ins', '% Exp/Vot', 'N°Panneau', 'Sexe', 'Nom', 'Prénom', 'Voix', '% Voix/Ins', '% Voix/Exp', 'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38', 'Unnamed: 39', 'Unnamed: 40', 'Unnamed: 41', 'Unnamed: 42', 'Unnamed: 43', 'Unnamed: 44', 'Unnamed: 45', 'Unnamed: 46', 'Unnamed: 47', 'Unnamed: 48', 'Unnamed: 49', 'Unnamed: 50', 'Unnamed: 51', 'Unnamed: 52', 'Unnamed: 53', 'Unnamed: 54', 'Unnamed: 55', 'Unnamed: 56', 'Unnamed: 57', 'Unnamed: 58', 'Unnamed: 59', 'Unnamed: 60', 'Unnamed: 61', 'Unnamed: 62', 'Unnamed: 63', 'Unnamed: 64', 'Unnamed: 65', 'Unnamed: 66', 'Unnamed: 67', 'Unnamed: 68', 'Unnamed: 69', 'Unnamed: 70', 'Unnamed: 71', 'Unnamed: 72', 'Unnamed: 73', 'Unnamed: 74', 'Unnamed: 75', 'Unnamed: 76', 'Unnamed: 77', 'Unnamed: 78', 'Unnamed: 79', 'Unnamed: 80', 'Unnamed: 81', 'Unnamed: 82', 'Unnamed: 83', 'Unnamed: 84', 'Unnamed: 85', 'Unnamed: 86', 'Unnamed: 87', 'Unnamed: 88', 'Unnamed: 89', 'Unnamed: 90', 'Unnamed: 91', 'Unnamed: 92', 'Unnamed: 93', 'Unnamed: 94', 'Unnamed: 95', 'Unnamed: 96', 'Unnamed: 97', 'Unnamed: 98', 'Unnamed: 99', 'Unnamed: 100', 'Unnamed: 101', 'Unnamed: 102']


# Construction du code INSEE
df["code_insee"] = df["Code du département"].astype(str) + df["Code de la commune"].astype(str).str.zfill(3)

# Repérage des blocs candidats (1er bloc nommé, puis tout le reste en blocs de 6 colonnes)
base_cols = list(df.columns[:26])  # jusqu'à % Voix/Exp inclus (pour le 1er candidat)
unnamed_cols = list(df.columns[26:])
# Indices des blocs candidats dans les colonnes "Unnamed"
candidate_blocks = []
i = 0
while i < len(unnamed_cols):
    bloc = unnamed_cols[i:i+7]
    if len(bloc) < 7:
        break
    candidate_blocks.append(bloc)
    i += 7

records = []

# On commence avec le 1er candidat (colonnes nommées)
for _, row in df.iterrows():
    # 1er candidat
    try:
        id_panneau1 = row["N°Panneau"]
        nom1 = str(row["Nom"]).strip()
        prenom1 = str(row["Prénom"]).strip()
        pct_exp1 = row["% Voix/Exp"]
        if id_panneau1 and nom1 and prenom1 and pd.notnull(pct_exp1):
            records.append({
                "code_insee": row["code_insee"],
                "libelle_commune": row["Libellé de la commune"],
                "pct_abstention": float(str(row["% Abs/Ins"]).replace(',', '.')),
                "id_panneau": row["N°Panneau"],
                "candidat": f"{nom1} {prenom1}",
                "pct_exp_vot": float(str(pct_exp1).replace(',', '.'))
            })
    except Exception as e:
        pass

    # Les candidats suivants
    for bloc in candidate_blocks:
        try:
            id_panneau = row[bloc[0]]
            nom = str(row[bloc[2]]).strip()
            prenom = str(row[bloc[3]]).strip()
            pct_exp = row[bloc[6]]
            if id_panneau and nom and prenom and pd.notnull(pct_exp) and nom.lower() != 'nan' and prenom.lower() != 'nan':
                records.append({
                    "code_insee": row["code_insee"],
                    "libelle_commune": row["Libellé de la commune"],
                    "pct_abstention": float(str(row["% Abs/Ins"]).replace(',', '.')),
                    "id_panneau": id_panneau,
                    "candidat": f"{nom} {prenom}",
                    "pct_exp_vot": float(str(pct_exp).replace(',', '.'))
                })
        except Exception as e:
            continue

df_long = pd.DataFrame(records)

# Vérification
print(df_long.dtypes)
#print(df_long.head(10))
display(df_long)

# Mapping des candidats vers leur orientation
orientation_map = {
    "ARTHAUD Nathalie":    "extreme_gauche",
    "POUTOU Philippe":     "extreme_gauche",
    "ROUSSEL Fabien":      "gauche",
    "MÉLENCHON Jean-Luc":  "gauche",
    "HIDALGO Anne":        "gauche",
    "JADOT Yannick":       "ecologiste",
    "MACRON Emmanuel":     "centre",
    "LASSALLE Jean":       "droite",
    "PÉCRESSE Valérie":    "droite",
    "DUPONT-AIGNAN Nicolas":"droite",
    "LE PEN Marine":       "extreme_droite",
    "ZEMMOUR Éric":        "extreme_droite"
}

df_long["orientation"] = df_long["candidat"].map(orientation_map)

# agrégation par orientation
df_agg = df_long.groupby(["code_insee", "libelle_commune", "orientation"]).agg({
    "pct_exp_vot": "sum",
    "pct_abstention": "first"  # abstention est la même par commune
}).reset_index()

display(df_agg)


['Code du département', 'Libellé du département', 'Code de la commune', 'Libellé de la commune', 'Etat saisie', 'Inscrits', 'Abstentions', '% Abs/Ins', 'Votants', '% Vot/Ins', 'Blancs', '% Blancs/Ins', '% Blancs/Vot', 'Nuls', '% Nuls/Ins', '% Nuls/Vot', 'Exprimés', '% Exp/Ins', '% Exp/Vot', 'N°Panneau', 'Sexe', 'Nom', 'Prénom', 'Voix', '% Voix/Ins', '% Voix/Exp', 'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38', 'Unnamed: 39', 'Unnamed: 40', 'Unnamed: 41', 'Unnamed: 42', 'Unnamed: 43', 'Unnamed: 44', 'Unnamed: 45', 'Unnamed: 46', 'Unnamed: 47', 'Unnamed: 48', 'Unnamed: 49', 'Unnamed: 50', 'Unnamed: 51', 'Unnamed: 52', 'Unnamed: 53', 'Unnamed: 54', 'Unnamed: 55', 'Unnamed: 56', 'Unnamed: 57', 'Unnamed: 58', 'Unnamed: 59', 'Unnamed: 60', 'Unnamed: 61', 'Unnamed: 62', 'Unnamed: 63', 'Unnamed: 64', 'Unnamed: 65', 'Unnamed: 66', 'Unnamed: 67', 'Unn

,code_insee,libelle_commune,pct_abstention,id_panneau,candidat,pct_exp_vot
0,77001,Achères-la-Forêt,17.17,1,ARTHAUD Nathalie,0.27
1,77001,Achères-la-Forêt,17.17,2,ROUSSEL Fabien,1.75
2,77001,Achères-la-Forêt,17.17,3,MACRON Emmanuel,27.69
3,77001,Achères-la-Forêt,17.17,4,LASSALLE Jean,2.02
4,77001,Achères-la-Forêt,17.17,5,LE PEN Marine,21.77
...,...,...,...,...,...,...
15199,95690,Wy-dit-Joli-Village,16.84,8,HIDALGO Anne,0.87
15200,95690,Wy-dit-Joli-Village,16.84,9,JADOT Yannick,8.66
15201,95690,Wy-dit-Joli-Village,16.84,10,PÉCRESSE Valérie,5.19
15202,95690,Wy-dit-Joli-Village,16.84,11,POUTOU Philippe,0.87


,code_insee,libelle_commune,orientation,pct_exp_vot,pct_abstention
0,77001,Achères-la-Forêt,centre,27.69,17.17
1,77001,Achères-la-Forêt,droite,13.44,17.17
2,77001,Achères-la-Forêt,ecologiste,6.05,17.17
3,77001,Achères-la-Forêt,extreme_droite,32.39,17.17
4,77001,Achères-la-Forêt,extreme_gauche,1.08,17.17
...,...,...,...,...,...
7597,95690,Wy-dit-Joli-Village,droite,10.81,16.84
7598,95690,Wy-dit-Joli-Village,ecologiste,8.66,16.84
7599,95690,Wy-dit-Joli-Village,extreme_droite,31.16,16.84
7600,95690,Wy-dit-Joli-Village,extreme_gauche,1.30,16.84


In [8]:
# Sauvegarde locale temporaire
df_agg.to_csv('testOK_IDF.csv', index=False)

In [9]:
from pandas_gbq import to_gbq

def export_to_bigquery_pandas_gbq(df, project_id, dataset_id, table_id):
    """
    Exporte un DataFrame vers BigQuery avec pandas-gbq

    Args:
        df: DataFrame à exporter
        project_id: ID du projet GCP
        dataset_id: ID du dataset BigQuery
        table_id: ID de la table BigQuery
    """

    # Configuration de la destination
    destination_table = f"{dataset_id}.{table_id}"

    try:
        # Export vers BigQuery
        to_gbq(
            df,
            destination_table=destination_table,
            project_id=project_id,
            if_exists='replace',  # Options: 'fail', 'replace', 'append'
            progress_bar=True,
            auth_local_webserver=False
        )

        print(f"✅ DataFrame exporté avec succès vers {project_id}.{destination_table}")
        print(f"📊 Nombre de lignes exportées : {len(df)}")
        print(f"📋 Nombre de colonnes : {len(df.columns)}")

    except Exception as e:
        print(f"❌ Erreur lors de l'export : {str(e)}")

df = df_agg
project_id = "mspr-data-ia"
dataset_id = "idf_silver_cleaned"
table_id = "faits_donnees_electorales"
export_to_bigquery_pandas_gbq(df, project_id, dataset_id, table_id)

100%|██████████| 1/1 [00:00<00:00, 9822.73it/s]

✅ DataFrame exporté avec succès vers mspr-data-ia.idf_silver_cleaned.faits_donnees_electorales
📊 Nombre de lignes exportées : 7602
📋 Nombre de colonnes : 5
